In [1]:
import os
from dataset_and_utils import (
    TokenEmbeddingsHandler,
    load_models
)
import torch
from diffusers import StableDiffusionXLPipeline

pretrained_model_name_or_path = "SG161222/RealVisXL_V5.0"
local_weights_cache = '/workspace/AI-avatar-headshot/output_cog/Itay/'
revision = None
weight_dtype = torch.float32
is_sdxl = True
device = "cuda"
(
    tokenizer_one,
    tokenizer_two,
    noise_scheduler,
    text_encoder_one,
    text_encoder_two,
    vae,
    unet,
) = load_models(pretrained_model_name_or_path, revision, device, weight_dtype, is_sdxl)
handler = TokenEmbeddingsHandler(
    [text_encoder_one, text_encoder_two], [tokenizer_one, tokenizer_two]
)
# handler.load_embeddings(os.path.join(local_weights_cache, "embeddings.pti"))
handler.load_embeddings("/workspace/AI-avatar-headshot/output_cog/Itay/embeddings.pti")
pipe = StableDiffusionXLPipeline(vae = vae , text_encoder = text_encoder_one , text_encoder_2 = text_encoder_two , unet = unet , tokenizer = tokenizer_one ,  tokenizer_2 = tokenizer_two , scheduler = noise_scheduler ).to('cuda')
pipe.load_lora_weights('/workspace/AI-avatar-headshot/training_sdxl_pti/ckpt/Itay_cog_lora.safetensors' , adapter_name = 'lora')
pipe.set_adapters("lora")


/workspace/miniconda3/envs/tan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
Fetching 2 files: 100%|██████████| 2/2 [02:11<00:00, 65.99s/it] 
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [2]:
prompt = "a <s0><s1> man wearing knitted red and black sweater, plain whitebox background, looking at the camera, detailed"
# for k, v in token_map.items():
#     prompt = prompt.replace(k, v)
# print(f"Prompt: {prompt}")
neg_prompt = "(worst quality, low quality, normal quality), disabled body, sketches, (manicure:1.2), lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, extra fingers, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry, momochrome, (ugly) , bad hand ,  bad leg , lost fingers, 4 fingers."
num_outputs = 3
width = 768
height = 1024
guidance_scale = 3.5
num_inference_steps = 25
seed = 1234
lora_scale = 0.7
generator = [
    torch.Generator(device="cuda").manual_seed(seed + i)
    for i in range(num_outputs)
]
sdxl_kwargs = {}
common_args = {
    "prompt": [prompt] * num_outputs if prompt is not None else None,
    "negative_prompt": [neg_prompt] * num_outputs if prompt is not None else None,
    "guidance_scale": guidance_scale,
    "generator": generator,
    "num_inference_steps": num_inference_steps,
    "max_sequence_length":256
}            
sdxl_kwargs["width"] = width
sdxl_kwargs["height"] = height
sdxl_kwargs["cross_attention_kwargs"] = {"scale": 1}
results = pipe(**common_args, **sdxl_kwargs).images
output_path = '/workspace/AI-avatar-headshot/output_cog/Itay/infer_results/'

os.makedirs(output_path, exist_ok=True)
for i , img in enumerate(results):
    img.save(f'{output_path}/{i}.png')

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:57<00:00,  2.31s/it]


In [6]:
prompt = "Skin texture, Closeup portrait photo of a stunning young <s1> man dressed as a highborn noble from game of thrones,f /2.8, Canon, 85mm,cinematic, high quality, looking at the camera, <s1> man"

neg_prompt = "(worst quality, low quality, normal quality), disabled body, sketches, (manicure:1.2), lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, extra fingers, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry, momochrome, (ugly) , bad hand ,  bad leg , lost fingers, 4 fingers."
num_outputs = 3
width = 768
height = 1024
guidance_scale = 6
num_inference_steps = 30
seed = 1234
lora_scale = 1
generator = [
    torch.Generator(device="cuda").manual_seed(seed + i)
    for i in range(num_outputs)
]
sdxl_kwargs = {}
common_args = {
    "prompt": [prompt] * num_outputs if prompt is not None else None,
    "negative_prompt": [neg_prompt] * num_outputs if prompt is not None else None,
    "guidance_scale": guidance_scale,
    "generator": generator,
    "num_inference_steps": num_inference_steps,
    "max_sequence_length":256
}            
sdxl_kwargs["width"] = width
sdxl_kwargs["height"] = height
sdxl_kwargs["cross_attention_kwargs"] = {"scale": 1}
results = pipe(**common_args, **sdxl_kwargs).images
output_path = '/workspace/AI-avatar-headshot/output_cog/Itay/infer_results/'

os.makedirs(output_path, exist_ok=True)
for i , img in enumerate(results):
    img.save(f'{output_path}/{i+6}.png')

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [01:09<00:00,  2.31s/it]


In [7]:
prompt = "beautiful young <s1> man, peach orange teal sweather, bright face, depth of field urban by Annie Leibovitz"

neg_prompt = "(worst quality, low quality, normal quality), disabled body, sketches, (manicure:1.2), lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, extra fingers, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry, momochrome, (ugly) , bad hand ,  bad leg , lost fingers, 4 fingers."
num_outputs = 3
width = 768
height = 1024
guidance_scale = 3.5
num_inference_steps = 20
seed = 1234
lora_scale = 0.7
generator = [
    torch.Generator(device="cuda").manual_seed(seed + i)
    for i in range(num_outputs)
]
sdxl_kwargs = {}
common_args = {
    "prompt": [prompt] * num_outputs if prompt is not None else None,
    "negative_prompt": [neg_prompt] * num_outputs if prompt is not None else None,
    "guidance_scale": guidance_scale,
    "generator": generator,
    "num_inference_steps": num_inference_steps,
    "max_sequence_length":256
}            
sdxl_kwargs["width"] = width
sdxl_kwargs["height"] = height
sdxl_kwargs["cross_attention_kwargs"] = {"scale": 1}
results = pipe(**common_args, **sdxl_kwargs).images
output_path = '/workspace/AI-avatar-headshot/output_cog/Itay/infer_results/'
for i , img in enumerate(results):
    img.save(f'{output_path}/{i+9}.png')

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:45<00:00,  2.30s/it]


: 

In [6]:
prompt = "a portrait of a ali01 man, close up photo of smiling, headshot for linkedin, professional, detailed, sharp focus, warm light, attractive, full background, directed, vivid colors, perfect composition, elegant, intricate, beautiful, highly saturated color, epic, stunning, gorgeous, cinematic, striking, rich deep detail, romantic, inspired, vibrant, illuminated, fancy, pretty, amazing, symmetry"

neg_prompt = "(worst quality, low quality, normal quality), disabled body, sketches, (manicure:1.2), lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, extra fingers, fewer digits, cropped, jpeg artifacts, signature, watermark, username, blurry, momochrome, (ugly) , bad hand ,  bad leg , lost fingers, 4 fingers."
num_outputs = 3
width = 768
height = 1024
guidance_scale = 3.5
num_inference_steps = 30
seed = 1234
lora_scale = 0.9
generator = [
    torch.Generator(device="cuda").manual_seed(seed + i)
    for i in range(num_outputs)
]
sdxl_kwargs = {}
common_args = {
    "prompt": [prompt] * num_outputs if prompt is not None else None,
    "negative_prompt": [neg_prompt] * num_outputs if prompt is not None else None,
    "guidance_scale": guidance_scale,
    "generator": generator,
    "num_inference_steps": num_inference_steps,
    "max_sequence_length":256
}            
sdxl_kwargs["width"] = width
sdxl_kwargs["height"] = height
sdxl_kwargs["cross_attention_kwargs"] = {"scale": 1}
results = pipe(**common_args, **sdxl_kwargs).images
for i , img in enumerate(results):
    img.save(f'{output_path}/{i+15}.png')

Token indices sequence length is longer than the specified maximum sequence length for this model (83 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', pretty, amazing, symmetry', ', pretty, amazing, symmetry', ', pretty, amazing, symmetry']
Token indices sequence length is longer than the specified maximum sequence length for this model (83 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', pretty, amazing, symmetry', ', pretty, amazing, symmetry', ', pretty, amazing, symmetry']
  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:54<00:00,  1.82s/it]


: 

In [5]:
# Script for converting a HF Diffusers saved pipeline to a Stable Diffusion checkpoint.
# *Only* converts the UNet, VAE, and Text Encoder.
# Does not convert optimizer state or any other thing.

import re

import torch
from safetensors.torch import load_file, save_file


# =================#
# UNet Conversion #
# =================#

unet_conversion_map = [
    # (stable-diffusion, HF Diffusers)
    ("time_embed.0.weight", "time_embedding.linear_1.weight"),
    ("time_embed.0.bias", "time_embedding.linear_1.bias"),
    ("time_embed.2.weight", "time_embedding.linear_2.weight"),
    ("time_embed.2.bias", "time_embedding.linear_2.bias"),
    ("input_blocks.0.0.weight", "conv_in.weight"),
    ("input_blocks.0.0.bias", "conv_in.bias"),
    ("out.0.weight", "conv_norm_out.weight"),
    ("out.0.bias", "conv_norm_out.bias"),
    ("out.2.weight", "conv_out.weight"),
    ("out.2.bias", "conv_out.bias"),
    # the following are for sdxl
    ("label_emb.0.0.weight", "add_embedding.linear_1.weight"),
    ("label_emb.0.0.bias", "add_embedding.linear_1.bias"),
    ("label_emb.0.2.weight", "add_embedding.linear_2.weight"),
    ("label_emb.0.2.bias", "add_embedding.linear_2.bias"),
]

unet_conversion_map_resnet = [
    # (stable-diffusion, HF Diffusers)
    ("in_layers.0", "norm1"),
    ("in_layers.2", "conv1"),
    ("out_layers.0", "norm2"),
    ("out_layers.3", "conv2"),
    ("emb_layers.1", "time_emb_proj"),
    ("skip_connection", "conv_shortcut"),
]

unet_conversion_map_layer = []
# hardcoded number of downblocks and resnets/attentions...
# would need smarter logic for other networks.
for i in range(3):
    # loop over downblocks/upblocks

    for j in range(2):
        # loop over resnets/attentions for downblocks
        hf_down_res_prefix = f"down_blocks.{i}.resnets.{j}."
        sd_down_res_prefix = f"input_blocks.{3*i + j + 1}.0."
        unet_conversion_map_layer.append((sd_down_res_prefix, hf_down_res_prefix))

        if i > 0:
            hf_down_atn_prefix = f"down_blocks.{i}.attentions.{j}."
            sd_down_atn_prefix = f"input_blocks.{3*i + j + 1}.1."
            unet_conversion_map_layer.append((sd_down_atn_prefix, hf_down_atn_prefix))

    for j in range(4):
        # loop over resnets/attentions for upblocks
        hf_up_res_prefix = f"up_blocks.{i}.resnets.{j}."
        sd_up_res_prefix = f"output_blocks.{3*i + j}.0."
        unet_conversion_map_layer.append((sd_up_res_prefix, hf_up_res_prefix))

        if i < 2:
            # no attention layers in up_blocks.0
            hf_up_atn_prefix = f"up_blocks.{i}.attentions.{j}."
            sd_up_atn_prefix = f"output_blocks.{3 * i + j}.1."
            unet_conversion_map_layer.append((sd_up_atn_prefix, hf_up_atn_prefix))

    if i < 3:
        # no downsample in down_blocks.3
        hf_downsample_prefix = f"down_blocks.{i}.downsamplers.0.conv."
        sd_downsample_prefix = f"input_blocks.{3*(i+1)}.0.op."
        unet_conversion_map_layer.append((sd_downsample_prefix, hf_downsample_prefix))

        # no upsample in up_blocks.3
        hf_upsample_prefix = f"up_blocks.{i}.upsamplers.0."
        sd_upsample_prefix = f"output_blocks.{3*i + 2}.{1 if i == 0 else 2}."
        unet_conversion_map_layer.append((sd_upsample_prefix, hf_upsample_prefix))
unet_conversion_map_layer.append(("output_blocks.2.2.conv.", "output_blocks.2.1.conv."))

hf_mid_atn_prefix = "mid_block.attentions.0."
sd_mid_atn_prefix = "middle_block.1."
unet_conversion_map_layer.append((sd_mid_atn_prefix, hf_mid_atn_prefix))
for j in range(2):
    hf_mid_res_prefix = f"mid_block.resnets.{j}."
    sd_mid_res_prefix = f"middle_block.{2*j}."
    unet_conversion_map_layer.append((sd_mid_res_prefix, hf_mid_res_prefix))


def convert_unet_state_dict(unet_state_dict):
    # buyer beware: this is a *brittle* function,
    # and correct output requires that all of these pieces interact in
    # the exact order in which I have arranged them.
    mapping = {k: k for k in unet_state_dict.keys()}
    for sd_name, hf_name in unet_conversion_map:
        mapping[hf_name] = sd_name
    for k, v in mapping.items():
        if "resnets" in k:
            for sd_part, hf_part in unet_conversion_map_resnet:
                v = v.replace(hf_part, sd_part)
            mapping[k] = v
    for k, v in mapping.items():
        for sd_part, hf_part in unet_conversion_map_layer:
            v = v.replace(hf_part, sd_part)
        mapping[k] = v
    new_state_dict = {sd_name: unet_state_dict[hf_name] for hf_name, sd_name in mapping.items()}
    return new_state_dict


# ================#
# VAE Conversion #
# ================#

vae_conversion_map = [
    # (stable-diffusion, HF Diffusers)
    ("nin_shortcut", "conv_shortcut"),
    ("norm_out", "conv_norm_out"),
    ("mid.attn_1.", "mid_block.attentions.0."),
]

for i in range(4):
    # down_blocks have two resnets
    for j in range(2):
        hf_down_prefix = f"encoder.down_blocks.{i}.resnets.{j}."
        sd_down_prefix = f"encoder.down.{i}.block.{j}."
        vae_conversion_map.append((sd_down_prefix, hf_down_prefix))

    if i < 3:
        hf_downsample_prefix = f"down_blocks.{i}.downsamplers.0."
        sd_downsample_prefix = f"down.{i}.downsample."
        vae_conversion_map.append((sd_downsample_prefix, hf_downsample_prefix))

        hf_upsample_prefix = f"up_blocks.{i}.upsamplers.0."
        sd_upsample_prefix = f"up.{3-i}.upsample."
        vae_conversion_map.append((sd_upsample_prefix, hf_upsample_prefix))

    # up_blocks have three resnets
    # also, up blocks in hf are numbered in reverse from sd
    for j in range(3):
        hf_up_prefix = f"decoder.up_blocks.{i}.resnets.{j}."
        sd_up_prefix = f"decoder.up.{3-i}.block.{j}."
        vae_conversion_map.append((sd_up_prefix, hf_up_prefix))

# this part accounts for mid blocks in both the encoder and the decoder
for i in range(2):
    hf_mid_res_prefix = f"mid_block.resnets.{i}."
    sd_mid_res_prefix = f"mid.block_{i+1}."
    vae_conversion_map.append((sd_mid_res_prefix, hf_mid_res_prefix))


vae_conversion_map_attn = [
    # (stable-diffusion, HF Diffusers)
    ("norm.", "group_norm."),
    # the following are for SDXL
    ("q.", "to_q."),
    ("k.", "to_k."),
    ("v.", "to_v."),
    ("proj_out.", "to_out.0."),
]


def reshape_weight_for_sd(w):
    # convert HF linear weights to SD conv2d weights
    if not w.ndim == 1:
        return w.reshape(*w.shape, 1, 1)
    else:
        return w


def convert_vae_state_dict(vae_state_dict):
    mapping = {k: k for k in vae_state_dict.keys()}
    for k, v in mapping.items():
        for sd_part, hf_part in vae_conversion_map:
            v = v.replace(hf_part, sd_part)
        mapping[k] = v
    for k, v in mapping.items():
        if "attentions" in k:
            for sd_part, hf_part in vae_conversion_map_attn:
                v = v.replace(hf_part, sd_part)
            mapping[k] = v
    new_state_dict = {v: vae_state_dict[k] for k, v in mapping.items()}
    weights_to_convert = ["q", "k", "v", "proj_out"]
    for k, v in new_state_dict.items():
        for weight_name in weights_to_convert:
            if f"mid.attn_1.{weight_name}.weight" in k:
                print(f"Reshaping {k} for SD format")
                new_state_dict[k] = reshape_weight_for_sd(v)
    return new_state_dict


# =========================#
# Text Encoder Conversion #
# =========================#


textenc_conversion_lst = [
    # (stable-diffusion, HF Diffusers)
    ("transformer.resblocks.", "text_model.encoder.layers."),
    ("ln_1", "layer_norm1"),
    ("ln_2", "layer_norm2"),
    (".c_fc.", ".fc1."),
    (".c_proj.", ".fc2."),
    (".attn", ".self_attn"),
    ("ln_final.", "text_model.final_layer_norm."),
    ("token_embedding.weight", "text_model.embeddings.token_embedding.weight"),
    ("positional_embedding", "text_model.embeddings.position_embedding.weight"),
]
protected = {re.escape(x[1]): x[0] for x in textenc_conversion_lst}
textenc_pattern = re.compile("|".join(protected.keys()))

# Ordering is from https://github.com/pytorch/pytorch/blob/master/test/cpp/api/modules.cpp
code2idx = {"q": 0, "k": 1, "v": 2}


def convert_openclip_text_enc_state_dict(text_enc_dict):
    new_state_dict = {}
    capture_qkv_weight = {}
    capture_qkv_bias = {}
    for k, v in text_enc_dict.items():
        if (
            k.endswith(".self_attn.q_proj.weight")
            or k.endswith(".self_attn.k_proj.weight")
            or k.endswith(".self_attn.v_proj.weight")
        ):
            k_pre = k[: -len(".q_proj.weight")]
            k_code = k[-len("q_proj.weight")]
            if k_pre not in capture_qkv_weight:
                capture_qkv_weight[k_pre] = [None, None, None]
            capture_qkv_weight[k_pre][code2idx[k_code]] = v
            continue

        if (
            k.endswith(".self_attn.q_proj.bias")
            or k.endswith(".self_attn.k_proj.bias")
            or k.endswith(".self_attn.v_proj.bias")
        ):
            k_pre = k[: -len(".q_proj.bias")]
            k_code = k[-len("q_proj.bias")]
            if k_pre not in capture_qkv_bias:
                capture_qkv_bias[k_pre] = [None, None, None]
            capture_qkv_bias[k_pre][code2idx[k_code]] = v
            continue

        relabelled_key = textenc_pattern.sub(lambda m: protected[re.escape(m.group(0))], k)
        new_state_dict[relabelled_key] = v

    for k_pre, tensors in capture_qkv_weight.items():
        if None in tensors:
            raise Exception("CORRUPTED MODEL: one of the q-k-v values for the text encoder was missing")
        relabelled_key = textenc_pattern.sub(lambda m: protected[re.escape(m.group(0))], k_pre)
        new_state_dict[relabelled_key + ".in_proj_weight"] = torch.cat(tensors)

    for k_pre, tensors in capture_qkv_bias.items():
        if None in tensors:
            raise Exception("CORRUPTED MODEL: one of the q-k-v values for the text encoder was missing")
        relabelled_key = textenc_pattern.sub(lambda m: protected[re.escape(m.group(0))], k_pre)
        new_state_dict[relabelled_key + ".in_proj_bias"] = torch.cat(tensors)

    return new_state_dict


def convert_openai_text_enc_state_dict(text_enc_dict):
    return text_enc_dict



model_dict = {
'text_encoder_one': text_encoder_one.state_dict(),
'text_encoder_two': text_encoder_two.state_dict(),
'vae': vae.state_dict(),
'unet': unet.state_dict(),
'text_encoder_one_embeddings': embeddings_one,
'text_encoder_two_embeddings': embeddings_two,
}
unet_state_dict = model_dict['unet']
vae_state_dict = model_dict['vae']
text_enc_dict = model_dict['text_encoder_one']
text_enc_2_dict = model_dict['text_encoder_two']
# Convert the UNet model
unet_state_dict = convert_unet_state_dict(unet_state_dict)
unet_state_dict = {"model.diffusion_model." + k: v for k, v in unet_state_dict.items()}

# Convert the VAE model
vae_state_dict = convert_vae_state_dict(vae_state_dict)
vae_state_dict = {"first_stage_model." + k: v for k, v in vae_state_dict.items()}

# Convert text encoder 1
text_enc_dict = convert_openai_text_enc_state_dict(text_enc_dict)
text_enc_dict = {"conditioner.embedders.0.transformer." + k: v for k, v in text_enc_dict.items()}

# Convert text encoder 2
text_enc_2_dict = convert_openclip_text_enc_state_dict(text_enc_2_dict)
text_enc_2_dict = {"conditioner.embedders.1.model." + k: v for k, v in text_enc_2_dict.items()}
# We call the `.T.contiguous()` to match what's done in
# https://github.com/huggingface/diffusers/blob/84905ca7287876b925b6bf8e9bb92fec21c78764/src/diffusers/loaders/single_file_utils.py#L1085
text_enc_2_dict["conditioner.embedders.1.model.text_projection"] = text_enc_2_dict.pop(
    "conditioner.embedders.1.model.text_projection.weight"
).T.contiguous()

# Put together new checkpoint
state_dict = {**unet_state_dict, **vae_state_dict, **text_enc_dict, **text_enc_2_dict}


state_dict = {k: v.half() for k, v in state_dict.items()}


save_file(state_dict, '/workspace/output_cog/Ali/updated_model.safetensors')
    

Reshaping encoder.mid.attn_1.q.weight for SD format
Reshaping encoder.mid.attn_1.k.weight for SD format
Reshaping encoder.mid.attn_1.v.weight for SD format
Reshaping encoder.mid.attn_1.proj_out.weight for SD format
Reshaping decoder.mid.attn_1.q.weight for SD format
Reshaping decoder.mid.attn_1.k.weight for SD format
Reshaping decoder.mid.attn_1.v.weight for SD format
Reshaping decoder.mid.attn_1.proj_out.weight for SD format


: 